<a href="https://colab.research.google.com/github/liddlecody/ds2002FinanceAPI/blob/main/FinanceAPIHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
import requests

## Task 1

In [3]:
#Enter 2 or more tickers
stocks=input()

MSFT


In [33]:
apikey="vn1t5x5hcw5aIWZzTjAFiayUoWaWecqs6BwEEaLB"

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stocks}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response.json()['quoteResponse']['result'])

df['regularMarketPrice'] = df['regularMarketPrice'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

#display results
print(df[['symbol', 'longName','regularMarketPrice']])

  symbol               longName regularMarketPrice
0   MSFT  Microsoft Corporation            $399.73


## Task 2.1

In [28]:
selected_symbol = input("\nEnter a stock symbol for additional module data: ").strip().upper()

module_choice = input("Choose a module: ").strip()


Enter a stock symbol for additional module data: AAPL
Choose a module: defaultKeyStatistics


In [29]:
summary_url = f"https://yfapi.net/v11/finance/quoteSummary/{selected_symbol}"

modules = module_choice +',financialData,summaryDetail'

querystring_module = {"modules": modules}


response_module = requests.request("GET", url=summary_url, headers=headers, params=querystring_module)
data_module = response_module.json()
result = data_module.get("quoteSummary", {}).get("result")

requested_module = result[0].get(module_choice, {})

financial_data = result[0].get("financialData", {})
roa = financial_data.get("returnOnAssets", {}).get("fmt", "N/A")
summary_detail = result[0].get("summaryDetail", {})
week_high = summary_detail.get("fiftyTwoWeekHigh", {}).get("fmt", "N/A")
week_low = summary_detail.get("fiftyTwoWeekLow", {}).get("fmt", "N/A")

data = {
    f"{module_choice}": requested_module,
    "returnOnAssets": roa,
    "fiftyTwoWeekHigh": week_high,
    "fiftyTwoWeekLow": week_low
}

df = pd.DataFrame([data])

print(f'Requested module and finacial data for {selected_symbol}')
df


Requested module and finacial data for AAPL


,defaultKeyStatistics,returnOnAssets,fiftyTwoWeekHigh,fiftyTwoWeekLow
0,"{'maxAge': 1, 'priceHint': {'raw': 2, 'fmt': '...",22.52%,260.10,164.08


## Task 2.2

In [31]:
trend_url = f"https://yfapi.net/v1/finance/trending/US"

querystring = {"region":'US'}

trending_response = requests.request("GET", url=trend_url, headers=headers, params=querystring)
trending_response = trending_response.json()

symbols_list = [quote["symbol"] for quote in trending_response["finance"]["result"][0]["quotes"]]


def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

all_stock_data = []

quote_url = "https://yfapi.net/v6/finance/quote"
headers = {'x-api-key': apikey}


for batch in chunk_list(symbols_list, 10):
    symbols_str = ",".join(batch)
    querystring = {"symbols": symbols_str}

    response_quote = requests.get(quote_url, headers=headers, params=querystring)
    data_quote = response_quote.json()
    results = data_quote.get("quoteResponse", {}).get("result", [])

    for item in results:
        ticker = item.get("symbol", "N/A")
        name = item.get("longName", item.get("shortName", "N/A"))
        current_price = item.get("regularMarketPrice", "N/A")
        week_high = item.get("fiftyTwoWeekHigh", "N/A")
        week_low = item.get("fiftyTwoWeekLow", "N/A")

        all_stock_data.append({
            "Ticker": ticker,
            "Name": name,
            "Current Price": current_price,
            "52 Week High": week_high,
            "52 Week Low": week_low
        })

df = pd.DataFrame(all_stock_data)

print(df)

     Ticker                                  Name  Current Price  \
0      NVDA                    NVIDIA Corporation         131.28   
1      SNOW                        Snowflake Inc.         166.19   
2       CRM                      Salesforce, Inc.         307.33   
3      IONQ                            IonQ, Inc.          29.93   
4      BYND                     Beyond Meat, Inc.           3.56   
5      MRNA                         Moderna, Inc.          33.58   
6      MARA                   MARA Holdings, Inc.          12.45   
7      TDOC                  Teladoc Health, Inc.          10.99   
8   BTC-USD                           Bitcoin USD       85064.94   
9        AI                           C3.ai, Inc.          26.44   
10      APP                  AppLovin Corporation         331.00   
11      NIO                              NIO Inc.           4.72   
12     NTNX                         Nutanix, Inc.          69.35   
13     ROOT                            Root, Inc